# OLED 屏幕控制

本教程介绍了如何通过JSON指令控制连接到ESP32模组的OLED显示屏。OLED显示屏是一种常用的显示设备，可以用于显示各种信息，如文本、图像等。

## OLED 屏幕基础信息

OLED显示屏通过I2C（IIC）接口与下位机ESP32模组进行通信。它可以显示自定义的文本内容，并支持多行显示。

产品上配有一个 OLED 显示屏，该显示屏通过 IIC 与下位机 ESP32 模组进行通信，当开机后会自动显示一些下位机的基础信息，上位机可以通过发送 JSON 指令来更改显示屏上显示的内容。

## OLED 屏幕控制 JSON 指令
- {"T":3,"lineNum":0,"Text":"putYourTextHere"}
    - 控制显示屏显示自定义内容
    - `lineNum` 是行数，一条JSON指令可以改变一行的内容。针对大部分产品使用的0.91英寸的OLED显示屏，lineNum的数值可以为0、1、2、3，共四行。     - `Text` 是你希望在这一行显示的文字内容。如果内容太多，会自动换行，但同时也会挤掉最后一行。
> lineNum 是行数，一条 JSON 指令可以改变一行的内容，下位机收到一条新的指令后，开机默认的 OLED 界面会消失，取而代之的是你新增加的内容，对于大部分产品使用的 0.91英寸的OLED显示屏，lineNum 的数值可以为 0、1、2、3，共四行；Text 是你希望在这一行显示的文字内容，如果你这一行的内容太多，会自动换行，但同时也会挤掉最后一行。

In [ ]:
from base_ctrl import BaseController

# 用于检测树莓派的函数
def is_raspberry_pi5():
    with open('/proc/cpuinfo', 'r') as file:
        for line in file:
            if 'Model' in line:
                if 'Raspberry Pi 5' in line:
                    return True
                else:
                    return False

# 根据树莓派的型号来确定 GPIO 串口设备名称
if is_raspberry_pi5():
    base = BaseController('/dev/ttyAMA0', 115200)
else:
    base = BaseController('/dev/serial0', 115200)

# 更改 OLED 上面的显示内容
base.send_command({"T":3,"lineNum":0,"Text":"this is line0"})
base.send_command({"T":3,"lineNum":1,"Text":"this is line1"})
base.send_command({"T":3,"lineNum":2,"Text":"this is line2"})
base.send_command({"T":3,"lineNum":3,"Text":"this is line3"})

运行上面的代码块后，OLED上面会显示出来四行文字：

this is line0

this is line1

this is line2

this is line3

## OLED 显示动态信息

在上面的教程中我们实现了在 OLED 屏幕上显示简单文字的方法，接下来我们写一个稍微复杂一些的例程，运行以下代码块后，OLED 屏幕上会显示当前的时间（但是由于树莓派的时间不一定是准确的所以和你当地的实际时间可能会有出入），本例程仅用于演示主程序中更新屏幕的方法，在产品主程序中，我们使用这样的方法来将产品的IP、运行状体等信息即时更新在 OLED 屏幕上。

In [ ]:
# 导入 datetime 模块中的 datetime 类，用于获取和处理当前日期和时间
from datetime import datetime
# 导入 time 模块，主要用于程序中的延时处理
import time

# 使用 while True 创建一个无限循环，使程序能够持续运行
while True:
    # 使用 datetime.now().strftime("%H:%M:%S") 获取当前时间，并将其格式化为 "小时:分钟:秒" 的形式
    current_time = datetime.now().strftime("%H:%M:%S")
    # 使用 base.send_command 方法发送一个包含当前时间的命令
    base.send_command({"T":3,"lineNum":0,"Text":current_time})
    # 使用 time.sleep(1) 让程序暂停1秒，这样可以确保每隔一秒更新一次时间，并发送一次命令
    time.sleep(1)

运行上面的代码块后，你可以看到 OLED 屏幕的第一行开始显示当前的时间，每秒钟更新一次，上面这条函数是无限循环的，你可以通过点击上面的 ■ 按键来终止程序运行。